In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
from datetime import datetime

In [3]:
tf.config.list_physical_devices('GPU')

[]

### About the mnist data set : http://yann.lecun.com/exdb/mnist/

In [4]:

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

template1 ='whole : {} | lev 1: {} | lev 2: {}'
print(template1.format(len(x_train), len(x_train[0]), len(x_train[0][0]), len(x_train[0][0]), "\n", ("=")*30))

print("This is whole:", "\n", ("-")*30, "\n", x_train, "\n", ("=")*30)
print("This is lev 1:", " \n", ("-")*30, "\n", x_train[0], "\n", ("=")*30)
print("This is lev 2:", " \n", ("-")*30, "\n", x_train[0][0], "\n", ("=")*30)
print("This is lev 3:", " \n", ("-")*30, "\n", x_train[0][0][0], "\n", ("=")*30)

whole : 60000 | lev 1: 28 | lev 2: 28
This is whole: 
 ------------------------------ 
 [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  

In [5]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

template1 ='whole: {} | lev 1: {} | lev 2: {} | lev 3: {}'
print(template1.format(len(x_train), len(x_train[0]), len(x_train[0][0]), len(x_train[0][0][0]), "\n", ("-")*30))

print("This is whole:", "\n", ("-")*30, "\n", x_train, "\n", ("=")*30)
print("This is lev 1:", " \n", ("-")*30, "\n", x_train[0], "\n", ("=")*30)
print("This is lev 2:", " \n", ("-")*30, "\n", x_train[0][0], "\n", ("=")*30)
print("This is lev 3:", " \n", ("-")*30, "\n", x_train[0][0][0], "\n", ("=")*30)
print("This is lev 4:", " \n", ("-")*30, "\n", x_train[0][0][0][0], "\n", ("=")*30)

whole: 60000 | lev 1: 28 | lev 2: 28 | lev 3: 1
This is whole: 
 ------------------------------ 
 [[[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [7]:
print(train_ds)
print(test_ds)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>
<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>


In [8]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [10]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [11]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)

In [12]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [13]:
def train_model(epochs):
    startTime = datetime.now()
    for epoch in range(epochs):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        for images, labels in train_ds:
            train_step(images, labels)
#             to suppress the float warning, use the line below
#             train_step(tf.cast(images, tf.float32), labels)

        for test_images, test_labels in test_ds:
            test_step(test_images, test_labels)
#             test_step(tf.cast(test_images, tf.float32), test_labels)

        template = 'Epoch {} :: Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print(template.format(epoch + 1,
                                train_loss.result(),
                                train_accuracy.result() * 100,
                                test_loss.result(),
                                test_accuracy.result() * 100))
    return print(f'Model trained in: (h:m:s.ms)\n {datetime.now() - startTime}')

### Kernel :: tf-cpu-env  :: using CPU

In [14]:
# with cpu
train_model(30)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1 :: Loss: 0.14366702735424042, Accuracy: 95.70000457763672, Test Loss: 0.06708572804927826, Test Accuracy: 97.83999633789062
Epoch 2 :: Loss: 0.04493744671344757, Accuracy: 98.60832977294922, Test Loss: 0.051962342113256454, Test Accuracy: 98.16999816894531
Epoch 3 :: Loss: 0.024612098932266235, Accuracy: 99.21666717529297, Test Loss: 0.056798361241817474, Test Accuracy: 98.18999481201172
Epoch 4 :: Loss: 0.014053307473659515, Accuracy: 99.52999877929688, Test Loss: 0.06460817903280258, Test Accu

In [15]:
break-here # on purpose so the cpu kernel upon switching won't run & override the gpu benchmark below

SyntaxError: invalid syntax (<ipython-input-15-78ee96d4cfce>, line 1)

### Kernel :: tf-gpu-env  :: using GPU

In [ ]:
# with gpu
train_model(30)